In [2]:
%matplotlib inline
import pandas as pd
import json
import geopandas as gpd
import plotly.express as px
import matplotlib.pyplot as plt

In [3]:
import plotly.io as pio
print(pio.renderers.default)

plotly_mimetype+notebook


In [20]:
# This is a destination x date matrix for Manhattan residents to NYC metro municipalities 
#("subdivision-place" geography - "subpl")

df_data = pd.read_csv(r'C:\Users\steve\GitHub\rp-covid-migration\data\dest_mnxco_dailyNorm_JanAug-19-20.csv')
df_data['county'] = df_data['county'].apply(lambda x: '{0:0>5}'.format(x))
df_data.head(10)

,county,2019-01-01,2019-01-02,2019-01-03,2019-01-04,2019-01-05,2019-01-06,2019-01-07,2019-01-08,2019-01-09,...,2020-08-22,2020-08-23,2020-08-24,2020-08-25,2020-08-26,2020-08-27,2020-08-28,2020-08-29,2020-08-30,2020-08-31
0,09001,469.505232,317.290634,302.272859,352.998177,511.774266,414.817458,268.672152,288.867908,249.082186,...,713.088096,769.981114,391.179922,331.337079,349.537318,429.288349,535.032367,590.293882,634.407265,346.727983
1,09005,113.050400,73.920757,60.454572,99.208638,187.881093,152.768795,20.234723,22.479993,16.244490,...,109.084298,103.771040,76.177143,72.029800,80.817877,96.741036,136.821254,181.304550,155.491977,113.142815
2,09009,228.407950,94.391120,82.543742,103.822993,171.744067,140.502687,61.828320,78.679975,67.143894,...,284.831222,294.709752,210.001853,154.349571,161.635754,189.451196,222.589802,339.418982,385.620102,202.562137
3,34003,1108.586063,726.697903,833.575539,996.700736,1247.161612,999.130221,723.953415,742.963762,868.538754,...,1464.557704,1135.255173,1004.714747,1109.258916,1058.714188,1060.120523,1027.180461,1547.413248,1335.157773,952.589511
4,34013,868.642357,829.049720,646.398884,677.156634,749.219083,753.808069,541.840910,510.295837,593.465383,...,731.268812,549.986510,496.180849,456.874730,497.029943,505.875003,434.969062,588.185690,717.336319,624.110369
5,34017,831.727941,765.364145,820.787072,880.188266,772.271978,781.685586,776.788525,744.087762,841.464603,...,937.316930,815.640371,611.475984,710.008026,666.747484,663.079187,741.285303,801.113126,887.340880,704.405270
6,34019,36.914416,22.744848,17.438819,19.611010,36.884632,26.762417,17.986420,7.867997,17.327456,...,54.542149,18.678787,20.588417,24.695931,38.388492,28.216136,10.210541,12.649155,39.391301,20.073725
7,34021,83.057437,51.175909,60.454572,62.293796,107.195961,100.359062,46.090202,53.951983,35.737879,...,161.606367,95.469356,51.471042,72.029800,52.531620,50.385956,77.600114,88.544082,126.466808,54.746524
8,34023,357.608407,203.566392,152.299018,214.567520,306.603502,248.667455,152.884572,162.979948,185.187191,...,478.758863,371.500322,234.707953,195.509456,198.003798,280.145918,306.316241,421.638487,416.718497,317.529837
9,34025,350.686954,161.488423,123.234320,199.570865,267.413581,252.012757,103.421917,118.019962,131.038889,...,870.654304,641.305025,413.827181,395.134902,361.659999,421.226596,461.516470,461.694143,644.773396,399.649622


In [58]:
###### transform table for animation, select only march - may
#list_date = df_data.columns.to_list()
#list_date = [x for x in list_date if x[0] == '3' or x[0] == '4' or x[0] == '5']

#pick only march 1 - march 20, because my animation won't render for more days
#remove this line if we can get 3 months to work
#list_date = list_date[:20]  


#melt for animation format
list_date = df_data.columns.tolist()
list_date = list_date[304:356] # march 1st to may 31st
#list_date = list_date[61:63] # march 1st to may 31st
df = pd.melt(df_data, id_vars=['county'],value_vars = list_date)
df = df.rename(columns={'county':'id'}) #rename so the join ids are the same
df['variable'] = df.variable.str.replace('-','/')
#df['val_pct'] = df['value']*100 # convert it into percentage
df.head(20)

,id,variable,value
0,09001,2020/03/01,711.970932
1,09005,2020/03/01,170.218338
2,09009,2020/03/01,250.417362
3,34003,2020/03/01,1497.594029
4,34013,2020/03/01,813.447249
5,34017,2020/03/01,1075.321614
6,34019,2020/03/01,37.644440
7,34021,2020/03/01,178.401912
8,34023,2020/03/01,384.627975
9,34025,2020/03/01,489.377721


In [59]:
len(df)

1612

In [60]:
df['value'].describe()

count      1612.000000
mean       2537.317683
std       10393.578222
min           0.000000
25%         163.599497
50%         303.139085
75%         869.740823
max      107206.939100
Name: value, dtype: float64

## Plotly

In [61]:
#read in geojson file
map_co = json.load(open('../data/shapefiles/nycmetro_county.geojson','r'))

#check properties
map_co['features'][0]['properties']

{'STATEFP': '09',
 'COUNTYFP': '009',
 'AFFGEOID': '0500000US09009',
 'GEOID': '09009',
 'NAME': 'New Haven',
 'ALAND': 1565645638,
 'AWATER': 667096946,
 'ATOTAL': 2232742584.0,
 'ATOTAL_mi': 862.06191,
 'ALAND_mi': 604.49891,
 'GEOID_Num': 9009.0}

In [62]:
# add feature id to geojson file
for feature in map_co['features']:
    feature['id'] = feature['properties']['GEOID']

In [63]:
map_co['features'][0]['id']

'09009'

In [64]:
fig = px.choropleth_mapbox(df, geojson = map_co,
                   locations ='id',
                   color = 'value',
                   hover_name = 'id',
                   mapbox_style="carto-positron",
                   animation_frame ='variable',
                   color_continuous_scale = "Blues",
                   labels={'value':'Trips per Device/100k'},
                   range_color = [0,10800.0])

#fig[“layout”].pop(“updatemenus”)
fig.add_annotation(text = 'Manhattan Counties',
                  align = 'right', x = 0.08, y = 0.95,
                  showarrow = False,
                  bordercolor = None,
                  bgcolor = None,
                  opacity = 0.95,
                  font = dict(family = "Droid Serif", color = 'black', size = 28 ))
#fig.add_annotation(text = 'Residents Away from Home',
 #                 align = 'right', x = 0.08, y = 0.90,
  #                showarrow = False,
   #               bordercolor = None,
    #              bgcolor = None,
     #             opacity = 0.95,
       #           font = dict(family = "Droid Serif", color = 'black', size = 28 ))
fig.add_annotation(text = 'From March 1st to May 31st',
                   align = 'right', x = 0.08, y = 0.85,
                  showarrow = False,
                  bordercolor = None,
                  bgcolor = None,
                  opacity = 0.95,
                  font = dict(family = "Droid Serif", color = 'black', size = 24 ))
fig.update_layout(mapbox_zoom = 7,
                 mapbox_center = {'lat':40.7, 'lon':-73.95},
                 margin = {'r':0,'t':0,'l':0,'b':0})
fig.write_html('county_dest.html', auto_open = True, auto_play = False, include_plotlyjs = 'cdn')